In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Linear(V2(..))
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices)

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [3]:
import Graphics.Dynamic.Plot.R2

type X = ℝ
type Y = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

wPlot' :: PointsWeb X (Shade' Y) -> DynamicPlottable
wPlot' wb = plot
            [ shapePlot $ fromVertices [p2 (x₁,y₁), p2 (x₂,y₂)]
            | ((x₁,Shade' y₁ _),(x₂,Shade' y₂ _)) <- edg ]
 where edg = webEdges wb

In [4]:
μ :: LocalLinear X Y +> Y
μ = arr.LinearFunction $ \d -> d$1

deq :: DifferentialEqn X Y
deq = constLinearODE μ

-- import Data.Type.Coercion
-- transformNorm (fmap (arr Coercion . transposeTensor) . blockVectSpan' $ (1 :: ℝ))
--     $ deq ((0^&4):±[1^&0, 0^&1])^.shadeNarrowness
--     :: Norm ℝ

In [5]:
skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    ([], []) -> []
    ([x], []) -> [x]
    (xs₀, []) -> [head xs₀, last xs₀]
    (x:_, xs') -> x : skipN n xs'

In [6]:
tf :: Needle X -> PointsWeb X (Shade' Y)
tf δx₀ = fromWebNodes euclideanMetric $ 
        (0, 1|±|[0.01]) : [(x, 1|±|[3]) | x<-[δx₀, 2*δx₀ .. 1.2] ]

forM_ [ iterateFilterDEqn_static AbortOnInconsistency deq (tf 0.05)
      , iterateFilterDEqn_adaptive euclideanMetric AbortOnInconsistency
                   deq (euclideanVolGoal 0.001) (tf 0.2)]
  $ \tfs -> do
    plotWindow
       [ plot ((1^&exp 1) :± [0.1^&0, 0^&0.1] :: Shade ℝ²) -- Euler's number as reference for x=1
       , plotLatest [plot st & legendName (show i) | (i,st) <- zip [0..] tfs]
         --  & plotDelay 2
       , plot ((0.4^&exp 0.4) :± [0.1^&0, 0^&0.1] :: Shade ℝ²) -- Euler's number as reference for x=1
       ]

Static resolution:
![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.gif)
Adaptive resolution:
![filtering the exponential function as a solution to an ODE (adaptive resolution)](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential-adaptive.gif)

In [7]:
type YZ = ℝ²

μ₂ :: LocalLinear X YZ +> YZ
μ₂ = arr.LinearFunction $ ($1) >>> \(V2 dy dz) -> V2 dz (-dy)

deq₂ :: DifferentialEqn X YZ
deq₂ = constLinearODE μ₂

In [8]:
tf₂ :: Needle X -> PointsWeb X (Shade' YZ)
tf₂ δx₀ = fromWebNodes euclideanMetric
        [(x, (1^&0)|±|[0^&δ, δ^&0]) | x<-(^2)<$>[0, sqrt δx₀ .. 2], let δ = 0.0001 + 2*x ]

forM_ [ iterateFilterDEqn_static AbortOnInconsistency deq₂ (tf₂ 0.004)
      , iterateFilterDEqn_adaptive euclideanMetric AbortOnInconsistency
              deq₂ (euclideanVolGoal 0.001) (tf₂ 0.01) ]
   $ \tfs₂ -> do
 plotWindow [ continFnPlot sin, continFnPlot cos
  , plotLatest [ plot [fmap fst tffacts, fmap snd tffacts] & legendName (show i)
               | (i,tf') <- zip [0..] tfs₂
               , let tffacts = fmap factoriseShade (coerceShade <$> tf' :: PointsWeb X (Shade' (Y,Y))) ] ]

![filtering the sin and cosine function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-trigonometric.png)
![filtering the sin and cosine function as a solution to an ODE (adaptive resolution)](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-trigonometric-adaptive.png)